# Flash Attention Deepdive

## Overview
Flash attention has evolved over the last 3 years with v1, v2 and v3. A table with the key differences is provided at the end of this notebook. 
- Analysis and Limitations of Vanilla attention
- Softmax and Online Softamx
- More on 
- Vanilla attention deepdive
- Flash Attention

Discuss Advance topics
- flex attention 
- deepseek attention methods
- Ohter customization from character ai etc. 

### Why
* Vanilla attentoin is O(N2) ... Elaborated in detail below in the section. 
* Prior attempts have mostly targeted at ideas like FLOPS reduction (sparse attention) using modified versions of attention without taking into account SW-HW co-design.
* More practically
    * Larger Sequence Lengths are required if we want to train models with increasing CTX lengths for a variety of use cases. 
* From a hardware perspective, vanilla attention is memory bound. 
    * HBM is an order of magnitude slower than SMEM. 

### How
Flash Attention V1 is inspired by the `DataMovement is all you need` paper. This paper critically analyzes the data movement and storage patterns of attention ops. The key takeaway is that vanilla attention is memory-bound (figure 1). 

* **IO Aware**: Takes into account the GPU memory hierarchy and compute architecture. Basically move beyond the mathematical / PyTorch formulation of attention and think critically about the target devices (Mi300x, H100, H200 etc) memory and compute properties. 
    * **Tiling**: To take full advantage of the various memory capacities and bandwidth (HBM, SMEM, L2 Cache, Registers, etc), the attention algorithm is re-organized in a way that doesn't change the end result (ie exact). It is just some "math tricks" to make the algo tiling compatible. **NOTE:** By default the attention function is not "tilable" due to the non-associate and non-distributive nature of the softmax function. More on this later. 

* **Activation Recomputation**
How do you get here ? Using the data movement paper + facts/research observations on Elementiwise vs matmul parts in terms of memory bound. 


In simpler terms, the flash attention algo provides a **fused kernel implememtation** of the original attention function $$O = dropout(mask(softmax(Q \dot K^T))V))$$. 

Formally, Dao et al. introduced FlashAttention, a novel tiling strategy for parallelizing attention that eliminates intermediate
reads/writes to slow global memory through fusing all of the attention operations into a single GPU kernel. Dao
[15] restructured the algorithm as FlashAttention-2 to also parallelize over the sequence length dimension and
perform the inner loop of the forward pass over blocks of the key and value matrices, thus improving the occupancy
and distribution of work on the GPU







 


## Revisit GPU Fundamentals

### GPU Memory Hierarchy
We'll take into account the realization that HBM r/w (2TB / sec) are an order of magnitude slower than SMEM r/w (20TB / sec)

| Memory Level         | H100                   | MI300X                 | Description                                                                                  |
|-----------------------|-------------------------|-------------------------|----------------------------------------------------------------------------------------------|
| Registers            | 256KB per SM, ~90 TB/s  | 256KB per CU, ~100 TB/s| Fastest memory, private to each thread. Stores variables and intermediate values for execution.|
| Shared Memory (SMEM) | upto 258KB per SM, ~50 TB/s  | 256KB per CU, ~60 TB/s | Programmable memory local to thread blocks. Used for data sharing and reuse within a block. Nvidia combines the SMEM and L1 cache into a unified memory region that the programmer can control allocation ratio for. For applications with high thread-level data sharing (e.g., matrix multiplication), the GPU can allocate more of this 256 KB as shared memory. |
| L1 Data Cache             | 128KB per SM, ~50 TB/s  | 128KB per CU, ~60 TB/s | Automatically caches frequently accessed local and global memory. Shared within an SM/CU.    |
| L2 Cache             | 50MB, 37.5 TB/s         | 256MB for 8 XCDs, 50 TB/s          | Last-level cache before accessing HBM. Shared across all SMs/CUs on the GPU. Also called as AMD Infinity Cache.               |
| HBM3                 | 80GB, 3.35 TB/s         | 192GB, 5.3 TB/s        | High Bandwidth Memory used for storing large datasets and models. Accessible by all SMs/CUs.|


<img src="diagrams/gpu_architecture.png" alt="Description" width="700" height="500">

## Vanilla Attention 
### Formulation
Remember that the IO unaware formulation of attention is as follows:

$\text{Let } Q, K, V \in \mathbb{R}^{N \times d} \text{ be the query, key and value input sequences}$
$ \text{associated to a single head, where } N \text{ is the sequence length and } d \text{ is the} $
$ \text{head dimension. Then the attention output } O \text{ is computed as:} $ 

\begin{align*}
& S = \alpha QK^\top \in \mathbb{R}^{N \times N}, && \\ 
& P = \text{softmax}(S) \in \mathbb{R}^{N \times N}, && \\ 
& O = PV \in \mathbb{R}^{N \times d}, &&
\end{align*}

Then give B Batches and H heads (MHA) for each of Q, K, V we get the following shapes: 
* Q: `[BatchSize (B), NumHeads (H), Seq Len (N), Embedding Dim (d)]`
* K: `[BatchSize (B), NumHeads (H), Seq Len (N), Embedding Dim (d)]`
* V: `[BatchSize (B), NumHeads (H), Seq Len (N), Embedding Dim (d)]`

You can parallelize the GeMM across BATCH and HEAD dimensions. 


### Limitation

#### I - Attention is $O (N^2)$ 
1. **Memory Bound**:  Standard computation would require 6 trips (3 loads and 3 stores) from the HBM and is memory bound. 
    * Load $Q$ & $K$ --> compute $X = Q \dot K^T$ --> Store $X$     
    * Load $X$ --> compute $A = softmax(X)$ --> Store $A$
    * Load $A$, $V$ --> compute $O = X  V$ --> Store $O$

    Add the distribution of time vs flops chart here. Link to glossary. 

2. **Materiallization**: If L is too large (for say a million context length) then it would **NOT be possible** at all to fit the entire $Q \dot K^T$ in the `SMEM`.
3. **Safe Softmax** Additionally in the real world we use  `Safe Softmax` operation. Safe Softmax **subtracts the MAX values to avoid overflow  and underflow** given a certain bit-width. Ex: IN fp16 (2bytes) the max value can only be 65556. Therefore if $x_{i}=12$ then  $e^{12}$ is too large to fit into the dynamic range of FP16. 
    * fp16 is with: `1 signed bit |  5 exponent bits |  10 mantissa` therefore very high precision but lower dynamic range 
    * bf16 is with: `1 signed bit |  8 exponent bits | 7 mantissa bit`  therefore very low precision but higher dynamic range

$$\text{Safe Softmax}(x_i) = \frac{e^{x_i - M}}{\sum_{j=1}^{n} e^{x_j - M}} \quad \text{where } M = \text{max} (x_i)$$


### Addressing the Limiations
1. Tiling
Matrix Mul Op have the nice property of **Associativity** & **Distributivity**. Therefore the $Q \dot K^T$ can be tiled achieving massive paralllism on GPUs. 
2. However, the subsequent operation `SOFTMAX` CANNOT be tiled. Additionally, using the 'Safe Softmax' operations requires us to scan the entire sequence length for the max value. 

#### Online Softmax
The online softmax (by Milakov et al) title `Online normalizer calculation for softmax` was introduced by Nvidia in 2018. 

1. Take care of overflows / underflow by subtracing the max value. IF the bit width is 16 (bf16, fp16).
2. 

The **implication / advantage of doing Online Softmax** is that equation (7) and equation (8) can be fused since you don't require $m_{N}$ any more and only require $m_{i}$ and $m_{i-1}$. All of this is hinged on the fact that $d_{N} == d^{`}_{N}$

$Q [k, :]  $  --> The Kth row of  

## Understanding the space-time complexity of Attention 
* Lower Bounds on IO access
* Parameterization
* 

## Python Autograd API
This section creates the python wrapper that integrates into Pytorch by inheriting the `torch.autograd.Function` class and implementing the `foward()` and `backward()` member functions. 

**FWD PASS SETUP & LAUNCH**

The broad recipe we follow: 
1. Asserts: Check if the shapes of Q, K, V tensors are good. Check if values are the type and shape they should be.
2. Allocate Input and Output buffers.
3. Decide how to parallelize the workload - reason about grids, warps, compute units, numblocks, block size w.r.t the workload under consideration
4. Launch FWD Triton Kernel using the `kernel_name[grid](<kernel params>)`
5. Save CTX needed for BWD. This context would not be necessary if were doing inference. The context is essentially data that the BWD pass will require. 
- LSE

$\text{Let } Q, K, V \in \mathbb{R}^{N \times d} \text{ be the query, key and value input sequences}$
$ \text{associated to a single head, where } N \text{ is the sequence length and } d \text{ is the} $
$ \text{head dimension. Then the attention output } O \text{ is computed as:} $ 

\begin{align*}
& S = \alpha QK^\top \in \mathbb{R}^{N \times N}, && \\ 
& P = \text{softmax}(S) \in \mathbb{R}^{N \times N}, && \\ 
& O = PV \in \mathbb{R}^{N \times d}, &&
\end{align*}

Then give B Batches and H heads (MHA) for each of Q, K, V we get the following shapes: 
* Q: `[BatchSize (B), NumHeads (H), Seq Len (N), Embedding Dim (d)]`
* K: `[BatchSize (B), NumHeads (H), Seq Len (N), Embedding Dim (d)]`
* V: `[BatchSize (B), NumHeads (H), Seq Len (N), Embedding Dim (d)]`

It's key to remember that the all the batches and the heads are to be treated independently [heads don't pass around info :P ]. Therefore the effective matrix becomes `[B*H, N, d]`where `B*H` acts as the batch dimension. 

In [ ]:
import triton
import torch 

def is_hip():
    return True if triton.runtime.driver.active.get_current_target().backend == 'hip' else False


class FusedAttention(torch.autograd.Function):
    @staticmethod
    def forward(ctx, q, k, v, causal:bool, softmax_scale: float):
        """
        the ctx object is used to build ctx that is required during the bwd pass
        we'll use ctx.save_for_backward(...) to do this. 
        """
        ### ------ 1. Check Inputs --------------
        HEAD_EMB_DIM_Q = HEAD_EMB_DIM_K = q.shape[-1] = k.shape[-1]  # q: [B, S, H, E]
        HEAD_EMB_DIM_V = v.shape[-1]

        assert HEAD_EMB_DIM_Q == HEAD_EMB_DIM_K == HEAD_EMB_DIM_V, "All heads must have the same embedding dimension"
        assert HEAD_EMB_DIM_K in {16, 32, 64, 128, 256}, "Only head size of 16, 32, 65, 128, 256 are supported"  
        # why are only these supported ??
        # - usually we prefer powers of 2 because of GPU SMs (ALUs)being a power of 2 for distirbuting computation. 
        # -  

        ### ------- 2. Allocate Buffers ----------
        o = torch.empty_like(q)      # whis the output the shape of q tensor ?
        stage = 3 if causal else 1   # causal requires special considerations such as masking. If masking is required then additional memory needs to be allocated for the boolean mask. 
        
        ### ------ 3. Grid, Warps, EUs, Blocks etc ---------
        # SM == EU; 
        # WARPS (32 threads) == WAVEFRONTS (64 threads) ; 
        # OCCUPANCY == WAVES_PER_EU == ACTIVE WARPS PER SM
        # How does this affect perf
            # - register pressure 
        if is_hip(): 
            # Increase the active warps count if ...
            # Decrease the warp count if ...
            waves_per_eu = 3 if HEAD_EMB_DIM_K <= 64 else 2 
            extra_kern_args = {"waves_per_eu": waves_per_eu, "allow_flush_denorm": True}

        # The grid can be 3D. 
        # Since Q.shape = [BATCH(0, B), NUM_HEADS(1, H), SEQ_LEN(2, L), EMB_DIM(3, d)] 
        # we know we can parallelize across the SEQ_LEN dim. since BATCH * NUM_HEADS is our BATCH DIM
        x_grid = triton.cdiv(q.shape[2], BLOCK_M)  # BLOCK_M is the tile size. If seq len = 1024 and BLOCK_M=128 then NUM_BLOCKS=8. Therefore each block will process BLOCK_M elements of the seq.
        y_grid = q.shape[0] * q.shape[1]           # BATCH * NUM_HEADS, since these are 100 % indepenedent.
        z_grid = 1 
        grid = (x_grid, y_grid, z_grid) # parallelization was done from inwards to outwards. 
        M = torch.empty((q.shape[0], q.shape[1], q.shape[2]), device=q.device, dtype=torch.float32)

        ### ------ 4. Launch ---------
        _attn_fwd[grid](
            q,
            k,
            v,
            softmax_scale,  # softmax scale. 
            M,  # causal mask tensor
            o,  #   
            q.stride(0), q.stride(1), q.stride(2), q.stride(3),  #  Batch, Head, Seq, Emb strides required for correct CUDA memory access. 
            k.stride(0), k.stride(1), k.stride(2), k.stride(3),  #  Batch, Head, Seq, Emb strides required for correct CUDA memory access.
            v.stride(0), v.stride(1), v.stride(2), v.stride(3),  #  Batch, Head, Seq, Emb strides required for correct CUDA memory access.
            o.stride(0), o.stride(1), o.stride(2), o.stride(3),  #  Batch, Head, Seq, Emb strides required for correct CUDA memory access.
            q.shape[0], q.shape[1],  #
            N_CTX=q.shape[2],  #
            HEAD_DIM=HEAD_DIM_K,  #
            STAGE=stage,  
            **extra_kern_args
        )

        ### Save context for BWD
        ctx.save_for_backward(q, k, v, o, M)
        ctx.grid = grid
        ctx.sm_scale = sm_scale
        ctx.HEAD_DIM = HEAD_DIM_K
        ctx.causal = causal

    
    @staticmethod
    def backward():
        """
        You can make mistakes while implemetning the bwd pass. To help with this we can use the .gradcheck() to use 
        infinite small internal [f(a+h) - f(a-h)] / 2h. In addition to that people use tools like PySolver
        """
        pass


# python api
def flash_attention_v2(
    batch_size: int,
    num_heads_q: int,
    num_heads_kv: int, 
    seq_len: int,
    emb_dim: int,
    causal:bool,
    mode: str,
    dropout: bool
):
    dtype = torch.float16
    device='cuda'

    q = torch.randn((batch_size, num_heads_q, seq_len, emb_dim), dtype=dtype, device=device) 
    k = torch.randn((batch_size, num_heads_kv, seq_len, emb_dim), dtype=dtype, device=device) 
    v = torch.randn((batch_size, num_heads_kv, seq_len, emb_dim), dtype=dtype, device=device) 
    
    if mode == 'fwd':
        q = q.to(torch.float8_e5m2) # why e5m2 for FWD pass ?
        k = k.to(torch.float8_e5m2)  # what is _nuz ?

    softmax_scale = 1.3  
    fn = lambda: FusedAttention.apply(q, k, v, causal, softmax_scale) # why use a lambda function here ?
    ms = triton.testing.do_bench(fn) # benchmark the op and report time in milli seconds. 

    

def main():
    BATCH_SIZE = 4
    NUM_HEADS_Q = 16
    NUM_HEAD_KV = 16
    SEQ_LEN = 4096
    EMB_DIM = 1024

    # Add support for QKV, QK and V, Q and K and V
    flash_attention_v2(
        batch_size=BATCH_SIZE,
        num_heads_q=NUM_HEADS_Q, 
        num_heads_kv=NUM_HEAD_KV,
        seq_len=SEQ_LEN,
        emb_dim=EMB_DIM,
        causal=False,
        mode='fwd',
        dropout=False
    )

## Triton Kernel

### Core Concepts
As shown above using the **online softmax** provides us with **eventual consistency**. Therefore we can now parallelize across 

* X: seq length
* Y: batch*head
* Z: 1 [no parallelism]

<mark>Each **CUDA THREAD BLOCK** gets a **UNIQUE PROGRAM ID**<mark>

<img src="diagrams/fa_triton_program_id_mapping.png" alt="Description" width="700" height="500">



BLOCK_M: Seq Tile size
BLOCK_N


Block level partial softmax is computed. For the partial softmax to converge to actual softmax we need to maintain 2 statistics - `m(x)`. We do this recursively over all the N/BLOCK_SIZE number of blocks,  accumulatating the results in the accumulator O.

**Notatation**
1. Given a shared memory size of $M$ we determined the block size to be  

    BLOCK_SIZE = ($B_c$,   $B_r$) where $B_c = \left\lceil\frac{M}{4d}\right\rceil$ and $B_r = \min\left(\left\lceil\frac{M}{4d}\right\rceil, d\right) $

2. Init Tensors
    * **Output tensor**: Allocate 0s for $O$ where $O = PV \in \mathbb{R}^{N \times d}$
    * m, l
3. Next let's chunk the Q, K, and V matrices into block
    * **row wise split**: divide the Q matrix into chunks ($Q_1, Q_2, .... Q_{Tr}$) where each chunk $Q_i$ is $B_r$ X $d$ size. We have a total of $T_r$ such chunks where $T_r = N / B_r$
    * **col wise split**: divide the K and V matrix into chunks ($K_1, K_2, .... K_{Tc}$) where each chunk $K_i$ is $B_c$ X $d$ size. We have a total of $T_c$ such chunks where $T_c = N / B_c$

4. Block wise output $O$ 

5. Implement **Fused Kernel** using Tiling and Online Softmax 
    * **Outer Loop**: Load $K_j$ and $V_j$ for each j in [1 .... $Tc$]
    * **Inner Loop**: Load $Q_i$ for each i in [1 ... $Tr$]
     

<img src="diagrams/flash_attention_algo_v1.png" alt="Description" width="900" height="500">


<table>
<tr>
<td width="45%" style="vertical-align: top;  background-color: #f6f8fa; ">

![Image Title](diagrams/inner_loop_attn.png)

</td>
<td width="55%" style="vertical-align: top; background-color: #f6f8fa; padding: 10px; border-radius: 5px; font-size: 0.85em">

```python
# Pseudocode
for b_c in range(0, N, B_c):  # Outer loop over K, V blocks
    # Load K,V blocks each of size (B_c x d) into SRAM 
    K_block = K[b_c:b_c+B_c, :]  # B_c tokens of the seq across all d dimension. Remember d is usually small and seq can be v large. 
    V_block = V[b_c:b_c+B_c, :]  # B_c × d block
    
    for b_r in range(0, N, B_r):  # Inner loop over Q blocks
        # Load Q block into SRAM
        Q_block = Q[b_r:b_r+B_r, :]  # B_r × d block
        
        # Compute attention for this block
        S = Q_block @ K_block.T    # B_r × B_c attention scores
        P = softmax(S)             # B_r × B_c attention weights
        O_block = P @ V_block      # B_r × d output block



1. Left: The GPU is a hierarchical computing and memory system. 
    * The HBM BW (gbps / seconds) is an order of magnitude (10x) slower that the the SMEM
    * However, the the SMEM memory is 100x smaller than the HBM in storage (HBM: 80GB H100, 192 GB MI300x vs SMEM: )
2. Middle: This is the core logic / kernel `_attention_fwd` of the flash attention algorithms in terms. This helps avoid the materialization of the N x N matrix 
    * The **outer for loop** loads the 
    
<img src="diagrams/inner_loop_attn_paper.png" alt="Description" width="1000" height="400">

### FWD KERNEL


In [1]:
@triton.jit
def _attn_fwd_inner(
    acc, 
    li, 
    mi, 
    q,  # [BLOCK_M, HEAD_DIM]
    K_block_ptr, V_block_ptr, 
    start_m, 
    qk_scale,  # softmax scale, default to sqrt(HEAD_DIM)
    BLOCK_M: tl.constexpr, HEAD_DIM: tl.constexpr, BLOCK_N: tl.constexpr,  #
    STAGE: tl.constexpr, 
    offs_m: tl.constexpr, offs_n: tl.constexpr,  #
    N_CTX: tl.constexpr,  # The operation is parallelized by dividing the sequence length (N_CTX) into blocks of size BLOCK_N.
    fp8_v: tl.constexpr
        
):
    """
    Load blocks of Q, K and V from HBM.
    """
    if STAGE == 1:
        lo, hi = 0, start_m * BLOCK_M
    elif STAGE == 2:
        lo, hi = start_m * BLOCK_M, (start_m + 1) * BLOCK_M
        lo = tl.multiple_of(lo, BLOCK_M)
    # causal = False, so keep the entire range of 0, N_CTX[no masking]
    else:
        lo, hi = 0, N_CTX
    
    # low, high marks the low/high index position over seq len. 


@triton.jit
def _attention_fwd(
    Q,                  # Q:  M, K matrix
    K,                  # Kt: K, N matrix
    V,                  # V:  K, N matrix
    softmax_scale, 
    OUT,                # O: M, N matrix
    stride_qz, stride_qh, stride_qm, stride_qk,  # jump by stride_qz to get to next batch and stride_qh to get the next head. 
    stride_kz, stride_kh, stride_kn, stride_kk,  # jump by stride_kk to get to next col in the Key matrix. 
    stride_vz, stride_vh, stride_vk, stride_vn,  # 
    stride_oz, stride_oh, stride_om, stride_on,  #
    B,                  # Batch Size 
    H,                  # Number of Heads
    N_CTX,      
    lse,         # 
    dropout_p, 
    is_causal, 
    HEAD_DIM: tl.constexpr,  # d
    BLOCK_M: tl.constexpr,   # Tile size in the QUERY SEQ DIMENSION
    BLOCK_N: tl.constexpr,   # Tile size for Key/Value dimension
    STAGE: tl.constexpr      #


):
    """
    Currently there is no support for
    1. Dropout
        - Requires a random number generator per thread. Philox Seed 
    2. SWA, GQA
    3. Variable sequence Lengths Layout (thd) --> (token, head,  dim_per_head)
    4. Context Parallelism
        - Due to intern token communication during attention, we need to all gather during fwd and reduce scatter during bwd

    """
    """
    Here we try to create a highly parallelizable "instance of the program". Therefore we must
    1. Define the program with the correct way for a program instance to know what data to work [which block, which tile, which batch which head & so on]
    2. 
    """
    # assume [BATCH(B/Z)=4, NUM_HEADS(H)=16, SEQ_LEN(L)=512, EMBED_DIM(d)=96]
    start_m = tl.program_id(0)    # get the current program instance along x axis. Achieve parallelism by tiling  along seq dim [X-axis]
    offset_bh = tl.program_id(1)  # get the current program instance along y axis.  Parallelism across the Batch * Head

    off_z = off_hz // H  # batch offset. if off_hz = 11 and H = 8 then batch_id = 11 // 8 = 1 (2nd batch) 
    off_h = off_hz % H   # head offset.  if off_hz = 11 and H = 8 then head_id = 11 % 8 = 3 (4th head) 

    # with very large tensor sizes, the offsets can be huge. Use int64 to avoid overflow. 
    # Get Q given a head and batch iie qkv_offset = batch_id * stride_batch_Q + head_id * stride_head_Q. 
    # Programs gets dispatched onto the BLOCKs(The abstraction for Triton)
    Q_block_ptr = tl.make_block_ptr(
        base = Q + qkv_offset,             # offset Q by qkv_offset to get to the current batch * head for the current threadblock. 
        shape = (N_CTX, HEAD_DIM),         # Shape of the parent tensor (seqlen tile size , head_dim), helps with bounds checking when reading
        strides = (stride_qm, stride_qk),  # The strides of the parent tensor [SeqLen X EmbedDim] ==> how to jump rows and cols of Q [mk] matrix. Helps with info contigous memory storage & access. 
        offsets = (start_m * BLOCK_M, 0)   # offsets to the block ==>  how to access the next block
        block_shape = (BLOCK_M, HEAD_DIM)  # Size of the block to load from HBM to SMEM / LDS [tile size of seqlen, d]
        order = (1, 0)                     # The order of the original data format [TN format]
    )

    K_block_ptr = tl.make_block_ptr(
        base = K + qkv_offset,             # The base pointer to the parent tensor
        shape = (N_CTX, HEAD_DIM),         # Shape of the parent tensor (seqlen tile size , head_dim)
        strides = (stride_kk, stride_kn),  # The strides of the parent tensor [EmbedDim X SeqLen] ==> How to jump rows and cols of K [kn] matrix 
        offsets = (0, 0)                   # offsets to the block # why are offserts 0, 0 ?
        block_shape = (HEAD_DIM, N_CTX)    # HEAD DIM, SEQ TILE SIZE
        order = (0, 1)                     # The order of the original data format
    )

    v_order: tl.constexpr = (0, 1) if V.dtype.element_ty == tl.float8e5 else (1, 0)
    V_block_ptr = tl.make_block_ptr(
        base = V + qkv_offset,             # The base pointer to the parent tensor
        shape = (N_CTX, HEAD_DIM),         # Shape of the parent tensor (seqlen tile size , head_dim)
        strides = (stride_vk, stride_vn),  # The strides of the parent tensor [EmbedDim X SeqLen] ==> How to jump rows and cols of V [kn] matrix 
        offsets=(0, 0),                    # offsets to the block
        block_shape = (BLOCK_N, HEAD_DIM)  # SEQ TILE SIZE , HEAD DIM
        order = v_order                    # The order of the original data format
    )
    O_block_ptr = tl.make_block_ptr(
        base=OUT + qkv_offset,
        shape=(N_CTX, HEAD_DIM),           # N X d
        strides=(stride_om, stride_on),    # The strides of the parent tensor [EmbedDim X SeqLen] ==> How to jump rows and cols of O [mn] matrix 
        offsets=(start_m * BLOCK_M, 0),    # Output BLOCKS
        block_shape=(BLOCK_M, HEAD_DIM),   # Each block is M x d
        order=(1, 0)
    )

    offs_m = start_m * BLOCK_M + tl.arange(0, BLOCK_M)

    # epilogue

    # Since we are dpoing everything block wise we need to have the right offset 
    # for storing m_ptrs
    # BLOCK_SIZE + JUMP ACROSS head * num batches * partial seqlen + offset_m 
    m_ptrs = M + offset_bh * N_CTX + offs_m
    tl.store(pointer=m_ptrs, value=m_i) # store the m_i block. 




SyntaxError: invalid syntax. Perhaps you forgot a comma? (4130513075.py, line 39)

### BWD KERNEL

#### PASS SETUP & LAUNCH
1. Asserts: Check if the shapes of Q, K, V tensors are good. Check if values are the type and shape they should be
2. Allocate Input and output buffers.
3. Decide how to parallelize the workload: grids, warps, compute units, numblocks, block size w.r.t the workload under consideration
4. Launch FWD Triton Kernel using the `grid[]()`
5. Save CTX needed for BWD

## Kernel Tuning
**Tile Size**
1. use a smaller tile size (16x16) for latency sensitive workloads. Use bigger for throughput sensitive. 
2. Smaller SMEM => Smaller tile size to fit in the fused op
3. Higher accuracy due to accumulation over smaller ranges. 

## Benchmarking
Compare across input types and sizes. 

1. GFLOPS per second
2. Data movement: GBPS per second
3. wall clock time (milliseconds)
4. GPU Level Metrics across compute and memory hierarchy
    * Compute
    * How many warps are active out of all the max warms - Occupancy
    * Thread Divergence
    * Cache hit rate
    * Register spills
    * Benchmarking: How many simulatenous blocks can run. What is the right balance for blocks vs tile size ? Does that/should that way across input sizes and types. large matrix small matrix, fp8, fp16, 

In [1]:
import torch

# Create a 2x3 tensor
tensor = torch.tensor([[1, 2, 3],
                      [4, 5, 6]])

print(tensor.stride())  # Output: (3, 1)

(3, 1)


In [7]:
tensor.t().is_contiguous()

False

In [8]:
tensor.is_contiguous()

True

## Glossary of Terms
1. Compute Bound vs Memory Bound
2. Arithmetic Intensity 
3. 